## A Large Data Workflow with Pandas 


##### Data Analysis of 8.2 Million Rows with Python and SQLite

This notebook explores a 3.9Gb CSV file containing NYC's 311 complaints since 2003. It's the most popular data set in [NYC's open data portal](https://nycopendata.socrata.com/data).

This notebook is a primer on out-of-memory data analysis with
- [pandas](http://pandas.pydata.org/): A library with easy-to-use data structures and data analysis tools. Also, interfaces to out-of-memory databases like SQLite.
- [IPython notebook](ipython.org/notebook.html): An interface for writing and sharing python code, text, and plots.
- [SQLite](https://www.sqlite.org/): An self-contained, server-less database that's easy to set-up and query from Pandas.
- [Plotly](https://plot.ly/python/): A platform for publishing beautiful, interactive graphs from Python to the web.

The dataset is too large to load into a Pandas dataframe. So, instead we'll perform out-of-memory aggregations with SQLite and load the result directly into a dataframe with Panda's `iotools`. It's pretty easy to stream a CSV into SQLite and SQLite requires no setup. The SQL query language is pretty intuitive coming from a Pandas mindset.

In [1]:
import plotly.tools as tls
tls.embed('https://plot.ly/~chris/7365')

In [2]:
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display

import plotly.plotly as py # interactive graphing
from plotly.graph_objs import Bar, Scatter, Marker, Layout 

#### Import the CSV data into SQLite

1. Load the CSV, chunk-by-chunk, into a DataFrame
2. Process the data a bit, strip out uninteresting columns
3. Append it to the SQLite database

In [3]:
display(pd.read_csv('311_100M.csv', nrows=2).head())
display(pd.read_csv('311_100M.csv', nrows=2).tail())

Unique Key            Created Date             Closed Date Agency  \
0    29300358  11/16/2014 11:46:00 PM  11/16/2014 11:46:00 PM   DSNY   
1    29299837  11/16/2014 02:24:35 AM  11/16/2014 02:24:35 AM    DOB   

               Agency Name     Complaint Type  \
0        BCC - Queens East  Derelict Vehicles   
1  Department of Buildings       Building/Use   

                                         Descriptor Location Type  \
0                              14 Derelict Vehicles        Street   
1  Illegal Conversion Of Residential Building/Space           NaN   

   Incident Zip         Incident Address  \
0         11432  80-25 PARSONS BOULEVARD   
1         10465    938 HUNTINGTON AVENUE   

                     ...                    Bridge Highway Name  \
0                    ...                                    NaN   
1                    ...                                    NaN   

  Bridge Highway Direction Road Ramp  Bridge Highway Segment  Garage Lot Name  \
0                      NaN       NaN                     NaN              NaN   
1                      NaN       NaN                     NaN              NaN   

  Ferry Direction Ferry Terminal Name   Latitude  Longitude  \
0             NaN                 NaN  40.719411 -73.808882   
1             NaN                 NaN  40.827862 -73.830641   

                                   Location  
0  (40.719410639341916, -73.80888158860446)  
1  (40.827862046105416, -73.83064067165407)  

[2 rows x 52 columns]

Unique Key            Created Date             Closed Date Agency  \
0    29300358  11/16/2014 11:46:00 PM  11/16/2014 11:46:00 PM   DSNY   
1    29299837  11/16/2014 02:24:35 AM  11/16/2014 02:24:35 AM    DOB   

               Agency Name     Complaint Type  \
0        BCC - Queens East  Derelict Vehicles   
1  Department of Buildings       Building/Use   

                                         Descriptor Location Type  \
0                              14 Derelict Vehicles        Street   
1  Illegal Conversion Of Residential Building/Space           NaN   

   Incident Zip         Incident Address  \
0         11432  80-25 PARSONS BOULEVARD   
1         10465    938 HUNTINGTON AVENUE   

                     ...                    Bridge Highway Name  \
0                    ...                                    NaN   
1                    ...                                    NaN   

  Bridge Highway Direction Road Ramp  Bridge Highway Segment  Garage Lot Name  \
0                      NaN       NaN                     NaN              NaN   
1                      NaN       NaN                     NaN              NaN   

  Ferry Direction Ferry Terminal Name   Latitude  Longitude  \
0             NaN                 NaN  40.719411 -73.808882   
1             NaN                 NaN  40.827862 -73.830641   

                                   Location  
0  (40.719410639341916, -73.80888158860446)  
1  (40.827862046105416, -73.83064067165407)  

[2 rows x 52 columns]

In [4]:
!wc -l < 311_100M.csv # Number of lines in dataset

 8281035


In [5]:
disk_engine = create_engine('sqlite:///311_8M.db') # Initializes database with filename 311_8M.db in current directory

In [6]:
start = dt.datetime.now()
chunksize = 20000
j = 0
index_start = 1

for df in pd.read_csv('311_100M.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) # Remove spaces from columns

    df['CreatedDate'] = pd.to_datetime(df['CreatedDate']) # Convert to datetimes
    df['ClosedDate'] = pd.to_datetime(df['ClosedDate'])

    df.index += index_start

    # Remove the un-interesting columns
    columns = ['Agency', 'CreatedDate', 'ClosedDate', 'ComplaintType', 'Descriptor',
               'CreatedDate', 'ClosedDate', 'TimeToCompletion',
               'City']

    for c in df.columns:
        if c not in columns:
            df = df.drop(c, axis=1)    

    
    j+=1
    print '{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize)

    df.to_sql('data', disk_engine, if_exists='append')
    index_start = df.index[-1] + 1

//anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1164: DtypeWarning:

Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.

//anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1164: DtypeWarning:

Columns (8,46) have mixed types. Specify dtype option on import or set low_memory=False.



6 seconds: completed 20000 rows
12 seconds: completed 40000 rows
18 seconds: completed 60000 rows
24 seconds: completed 80000 rows
30 seconds: completed 100000 rows
37 seconds: completed 120000 rows
43 seconds: completed 140000 rows
49 seconds: completed 160000 rows
55 seconds: completed 180000 rows
62 seconds: completed 200000 rows
68 seconds: completed 220000 rows
74 seconds: completed 240000 rows
81 seconds: completed 260000 rows
87 seconds: completed 280000 rows
99 seconds: completed 300000 rows
108 seconds: completed 320000 rows
116 seconds: completed 340000 rows
123 seconds: completed 360000 rows
131 seconds: completed 380000 rows
138 seconds: completed 400000 rows
149 seconds: completed 420000 rows
158 seconds: completed 440000 rows
164 seconds: completed 460000 rows
171 seconds: completed 480000 rows
177 seconds: completed 500000 rows
184 seconds: completed 520000 rows
190 seconds: completed 540000 rows
198 seconds: completed 560000 rows
204 seconds: completed 580000 rows
210 s

###### Preview the table

In [7]:
df = pd.read_sql_query('SELECT * FROM data LIMIT 3', disk_engine)
df.head()

index                 CreatedDate                  ClosedDate Agency  \
0      1  2014-11-16 23:46:00.000000  2014-11-16 23:46:00.000000   DSNY   
1      2  2014-11-16 02:24:35.000000  2014-11-16 02:24:35.000000    DOB   
2      3  2014-11-16 02:17:12.000000  2014-11-16 02:50:48.000000   NYPD   

       ComplaintType                                        Descriptor  \
0  Derelict Vehicles                              14 Derelict Vehicles   
1       Building/Use  Illegal Conversion Of Residential Building/Space   
2    Illegal Parking                                  Blocked Sidewalk   

       City  
0   Jamaica  
1     BRONX  
2  BROOKLYN

###### Select just a couple of columns

In [8]:
df = pd.read_sql_query('SELECT Agency, Descriptor FROM data LIMIT 3', disk_engine)
df.head()

Agency                                        Descriptor
0   DSNY                              14 Derelict Vehicles
1    DOB  Illegal Conversion Of Residential Building/Space
2   NYPD                                  Blocked Sidewalk

###### `LIMIT` the number of rows that are retrieved

In [9]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'LIMIT 10', disk_engine)
df

ComplaintType                                        Descriptor  \
0        Derelict Vehicles                              14 Derelict Vehicles   
1             Building/Use  Illegal Conversion Of Residential Building/Space   
2          Illegal Parking                                  Blocked Sidewalk   
3  Noise - Street/Sidewalk                                  Loud Music/Party   
4          Illegal Parking                      Commercial Overnight Parking   
5  Noise - Street/Sidewalk                                      Loud Talking   
6                  Traffic                               Congestion/Gridlock   
7       Noise - Commercial                                  Loud Music/Party   
8       Noise - Commercial                                  Loud Music/Party   
9       Noise - Commercial                                  Loud Music/Party   

  Agency  
0   DSNY  
1    DOB  
2   NYPD  
3   NYPD  
4   NYPD  
5   NYPD  
6   NYPD  
7   NYPD  
8   NYPD  
9   NYPD

###### Filter rows with `WHERE`

In [10]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'WHERE Agency = "NYPD" '
                       'LIMIT 10', disk_engine)
df.head()

ComplaintType                    Descriptor Agency
0          Illegal Parking              Blocked Sidewalk   NYPD
1  Noise - Street/Sidewalk              Loud Music/Party   NYPD
2          Illegal Parking  Commercial Overnight Parking   NYPD
3  Noise - Street/Sidewalk                  Loud Talking   NYPD
4                  Traffic           Congestion/Gridlock   NYPD

###### Filter multiple values in a column with `WHERE` and `IN`

In [11]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'WHERE Agency IN ("NYPD", "DOB")'
                       'LIMIT 10', disk_engine)
df.head()

ComplaintType                                        Descriptor  \
0             Building/Use  Illegal Conversion Of Residential Building/Space   
1          Illegal Parking                                  Blocked Sidewalk   
2  Noise - Street/Sidewalk                                  Loud Music/Party   
3          Illegal Parking                      Commercial Overnight Parking   
4  Noise - Street/Sidewalk                                      Loud Talking   

  Agency  
0    DOB  
1   NYPD  
2   NYPD  
3   NYPD  
4   NYPD

###### Find the unique values in a column with `DISTINCT`

In [12]:
df = pd.read_sql_query('SELECT DISTINCT City FROM data', disk_engine)
df.head()

City
0        Jamaica
1          BRONX
2       BROOKLYN
3       NEW YORK
4  STATEN ISLAND

###### Query value counts with `COUNT(*)` and `GROUP BY`

In [13]:
df = pd.read_sql_query('SELECT Agency, COUNT(*) as `num_complaints`'
                       'FROM data '
                       'GROUP BY Agency ', disk_engine)

df.head()

Agency  num_complaints
0  3-1-1           22029
1    ACS               2
2    AJC               2
3    ART               3
4    CAU               7

###### Order the results with `ORDER` and `-`
Housing and Development Dept receives the most complaints

In [14]:
df = pd.read_sql_query('SELECT Agency, COUNT(*) as `num_complaints`'
                       'FROM data '
                       'GROUP BY Agency '
                       'ORDER BY -num_complaints', disk_engine)

py.iplot([Bar(x=df.Agency, y=df.num_complaints)], filename='311/most common complaints by agency')

###### Heat / Hot Water is the most common complaint

In [15]:
df = pd.read_sql_query('SELECT ComplaintType, COUNT(*) as `num_complaints`, Agency '
                       'FROM data '
                       'GROUP BY `ComplaintType` '
                       'ORDER BY -num_complaints', disk_engine)


most_common_complaints = df # used later
py.iplot({
    'data': [Bar(x=df['ComplaintType'], y=df.num_complaints)],
    'layout': { 
        'margin': {'b': 150}, # Make the bottom margin a bit bigger to handle the long text
        'xaxis': {'tickangle': 40}} # Angle the labels a bit
    }, filename='311/most common complaints by complaint type')

*This graph is interactive. Click-and-drag horizontally to zoom, shift-click to pan, double click to autoscale*

##### What's the most common complaint in each city?

First, let's see how many cities are recorded in the dataset

In [16]:
len(pd.read_sql_query('SELECT DISTINCT City FROM data', disk_engine))

1758

Yikes - let's just plot the 10 most complained about cities

In [17]:
df = pd.read_sql_query('SELECT City, COUNT(*) as `num_complaints` '
                                'FROM data '
                                'GROUP BY `City` '
                       'ORDER BY -num_complaints '
                       'LIMIT 10 ', disk_engine)
df

City  num_complaints
0       BROOKLYN         2441941
1       NEW YORK         1544421
2          BRONX         1470746
3           None          654158
4  STATEN ISLAND          408095
5        JAMAICA          141940
6       FLUSHING          112519
7        ASTORIA           86051
8      RIDGEWOOD           63400
9        Jamaica           54876

Flushing and FLUSHING, Jamaica and JAMAICA... the complaints are case sensitive.

###### Perform case insensitive queries with `GROUP BY` with `COLLATE NOCASE`

In [18]:
df = pd.read_sql_query('SELECT City, COUNT(*) as `num_complaints` '
                        'FROM data '
                        'GROUP BY `City` '
                       'COLLATE NOCASE '
                       'ORDER BY -num_complaints '
                       'LIMIT 11 ', disk_engine)
df

City  num_complaints
0        BROOKLYN         2441941
1        NEW YORK         1544423
2           BRONX         1470746
3            None          654158
4   STATEN ISLAND          408095
5         JAMAICA          196816
6        FLUSHING          149625
7         ASTORIA          116103
8       RIDGEWOOD           86237
9        WOODSIDE           60148
10   FAR ROCKAWAY           59552

In [19]:
cities = list(df.City)
cities.remove(None)

In [20]:
traces = [] # the series in the graph - one trace for each city

for city in cities:
    df = pd.read_sql_query('SELECT ComplaintType, COUNT(*) as `num_complaints` '
                           'FROM data '
                           'WHERE City = "{}" COLLATE NOCASE '
                           'GROUP BY `ComplaintType` '
                           'ORDER BY -num_complaints'.format(city), disk_engine)

    traces.append(Bar(x=df['ComplaintType'], y=df.num_complaints, name=city.capitalize()))

In [21]:
py.iplot({'data': traces, 'layout': Layout(barmode='stack', xaxis={'tickangle': 40}, margin={'b': 150})}, filename='311/complaints by city stacked')

*You can also `click` on the legend entries to hide/show the traces. Click-and-drag to zoom in and shift-drag to pan.*

Now let's normalize these counts. This is super easy now that this data has been reduced into a dataframe.

In [22]:
for trace in traces:
    trace['y'] = 100.*trace['y']/sum(trace['y'])

In [23]:
py.iplot({'data': traces, 
          'layout': Layout(
                barmode='group',
                xaxis={'tickangle': 40, 'autorange': False, 'range': [-0.5, 16]},
                yaxis={'title': 'Percent of Complaints by City'},
                margin={'b': 150},
                title='Relative Number of 311 Complaints by City')
         }, filename='311/relative complaints by city', validate=False)

- New York is loud
- Staten Island is moldy, wet, and vacant
- Flushing's muni meters are broken 
- Trash collection is great in the Bronx
- Woodside doesn't like its graffiti

Click and drag to pan across the graph and see more of the complaints. 

### Part 2: SQLite time series with Pandas

######  Filter SQLite rows with timestamp strings: `YYYY-MM-DD hh:mm:ss`

In [24]:
df = pd.read_sql_query('SELECT ComplaintType, CreatedDate, City '
                       'FROM data '
                       'WHERE CreatedDate < "2014-11-16 23:47:00" '
                       'AND CreatedDate > "2014-11-16 23:45:00"', disk_engine)

df

ComplaintType                 CreatedDate     City
0  Derelict Vehicles  2014-11-16 23:46:00.000000  Jamaica

######  Pull out the hour unit from timestamps with `strftime`


In [25]:
df = pd.read_sql_query('SELECT CreatedDate, '
                              'strftime(\'%H\', CreatedDate) as hour, '
                              'ComplaintType '
                       'FROM data '
                       'LIMIT 5 ', disk_engine)
df.head()

CreatedDate hour            ComplaintType
0  2014-11-16 23:46:00.000000   23        Derelict Vehicles
1  2014-11-16 02:24:35.000000   02             Building/Use
2  2014-11-16 02:17:12.000000   02          Illegal Parking
3  2014-11-16 02:15:13.000000   02  Noise - Street/Sidewalk
4  2014-11-16 02:14:01.000000   02          Illegal Parking

######  Count the number of complaints (rows) per hour with `strftime`, `GROUP BY`, and `count(*)`

In [26]:
df = pd.read_sql_query('SELECT CreatedDate, '
                               'strftime(\'%H\', CreatedDate) as hour,  '
                               'count(*) as `Complaints per Hour`'
                       'FROM data '
                       'GROUP BY hour', disk_engine)

df.head()

CreatedDate hour  Complaints per Hour
0  2003-02-26 00:47:27.000000   00              3178595
1  2003-02-26 01:36:31.000000   01                71993
2  2003-03-04 02:00:46.000000   02                56362
3  2003-02-25 03:07:01.000000   03                33396
4  2003-03-04 04:32:11.000000   04                30434

In [27]:
py.iplot({
    'data': [Bar(x=df['hour'], y=df['Complaints per Hour'])],
    'layout': Layout(xaxis={'title': 'Hour in Day'},
                     yaxis={'title': 'Number of Complaints'})}, filename='311/complaints per hour')

######  Filter noise complaints by hour

In [28]:
df = pd.read_sql_query('SELECT CreatedDate, '
                               'strftime(\'%H\', CreatedDate) as `hour`,  '
                               'count(*) as `Complaints per Hour`'
                       'FROM data '
                       'WHERE ComplaintType IN ("Noise", '
                                               '"Noise - Street/Sidewalk", '
                                               '"Noise - Commercial", '
                                               '"Noise - Vehicle", '
                                               '"Noise - Park", '
                                               '"Noise - House of Worship", '
                                               '"Noise - Helicopter", '
                                               '"Collection Truck Noise") '
                       'GROUP BY hour', disk_engine)

display(df.head(n=2))

py.iplot({
    'data': [Bar(x=df['hour'], y=df['Complaints per Hour'])],
    'layout': Layout(xaxis={'title': 'Hour in Day'},
                     yaxis={'title': 'Number of Complaints'},
                     title='Number of Noise Complaints in NYC by Hour in Day'
                    )}, filename='311/noise complaints per hour')

CreatedDate hour  Complaints per Hour
0  2004-08-19 00:54:43.000000   00                41373
1  2008-08-29 01:07:39.000000   01                34588

######  Segregate complaints by hour

In [29]:
complaint_traces = {} # Each series in the graph will represent a complaint
complaint_traces['Other'] = {}

for hour in range(1, 24):
    hour_str = '0'+str(hour) if hour < 10 else str(hour)
    df = pd.read_sql_query('SELECT  CreatedDate, '
                                   'ComplaintType ,'
                                   'strftime(\'%H\', CreatedDate) as `hour`,  '
                                   'COUNT(*) as num_complaints '
                           'FROM data '
                           'WHERE hour = "{}" '
                           'GROUP BY ComplaintType '
                           'ORDER BY -num_complaints'.format(hour_str), disk_engine)
    
    complaint_traces['Other'][hour] = sum(df.num_complaints)
    
    # Grab the 7 most common complaints for that hour
    for i in range(7):
        complaint = df.get_value(i, 'ComplaintType')
        count = df.get_value(i, 'num_complaints')
        complaint_traces['Other'][hour] -= count
        if complaint in complaint_traces:
            complaint_traces[complaint][hour] = count
        else:
            complaint_traces[complaint] = {hour: count}

In [30]:
traces = []
for complaint in complaint_traces:
    traces.append({
        'x': range(25),
        'y': [complaint_traces[complaint].get(i, None) for i in range(25)],
        'name': complaint,
        'type': 'bar'
    })

py.iplot({
    'data': traces, 
    'layout': {
        'barmode': 'stack',
        'xaxis': {'title': 'Hour in Day'},
        'yaxis': {'title': 'Number of Complaints'},
        'title': 'The 7 Most Common 311 Complaints by Hour in a Day'
    }}, filename='311/most common complaints by hour')

##### Aggregated time series

First, create a new column with timestamps rounded to the previous 15 minute interval

In [31]:
minutes = 15
seconds = 15*60

df = pd.read_sql_query('SELECT CreatedDate, '
                               'datetime(('
                                   'strftime(\'%s\', CreatedDate) / {seconds}) * {seconds}, \'unixepoch\') interval '
                       'FROM data '
                       'LIMIT 10 '.format(seconds=seconds), disk_engine)

display(df.head())

CreatedDate             interval
0  2014-11-16 23:46:00.000000  2014-11-16 23:45:00
1  2014-11-16 02:24:35.000000  2014-11-16 02:15:00
2  2014-11-16 02:17:12.000000  2014-11-16 02:15:00
3  2014-11-16 02:15:13.000000  2014-11-16 02:15:00
4  2014-11-16 02:14:01.000000  2014-11-16 02:00:00

Then, `GROUP BY` that interval and `COUNT(*)`

In [32]:
minutes = 15
seconds = minutes*60

df = pd.read_sql_query('SELECT datetime(('
                                   'strftime(\'%s\', CreatedDate) / {seconds}) * {seconds}, \'unixepoch\') interval ,'
                               'COUNT(*) as "Complaints / interval"'
                       'FROM data '
                       'GROUP BY interval '
                       'ORDER BY interval '
                       'LIMIT 500'.format(seconds=seconds), disk_engine)

display(df.head())
display(df.tail())

interval  Complaints / interval
0  2003-02-24 09:15:00                      1
1  2003-02-24 09:30:00                      2
2  2003-02-24 09:45:00                      2
3  2003-02-24 10:00:00                      2
4  2003-02-24 10:15:00                      1

interval  Complaints / interval
495  2003-03-13 07:30:00                      2
496  2003-03-13 08:45:00                      1
497  2003-03-13 09:00:00                      1
498  2003-03-13 09:15:00                      1
499  2003-03-13 09:30:00                      2

In [33]:
py.iplot(
    {
        'data': [{
            'x': df.interval,
            'y': df['Complaints / interval'],
            'type': 'bar'
        }],
        'layout': {
            'title': 'Number of 311 Complaints per 15 Minutes'
        }
}, filename='311/complaints per 15 minutes')

In [34]:
hours = 24
minutes = hours*60
seconds = minutes*60

df = pd.read_sql_query('SELECT datetime(('
                                   'strftime(\'%s\', CreatedDate) / {seconds}) * {seconds}, \'unixepoch\') interval ,'
                               'COUNT(*) as "Complaints / interval"'
                       'FROM data '
                       'GROUP BY interval '
                       'ORDER BY interval'.format(seconds=seconds), disk_engine)

In [35]:
py.iplot(
    {
        'data': [{
            'x': df.interval,
            'y': df['Complaints / interval'],
            'type': 'bar'
        }],
        'layout': {
            'title': 'Number of 311 Complaints per Day'
        }
}, filename='311/complaints per day')

### Learn more

- Find more open data sets on [Data.gov](https://data.gov) and [NYC Open Data](https://nycopendata.socrata.com)
- Learn how to setup [MySql with Pandas and Plotly](http://moderndata.plot.ly/graph-data-from-mysql-database-in-python/)
- Add [interactive widgets to IPython notebooks](http://moderndata.plot.ly/widgets-in-ipython-notebook-and-plotly/) for customized data exploration
- Big data workflows with [HDF5 and Pandas](http://stackoverflow.com/questions/14262433/large-data-work-flows-using-pandas)
- [Interactive graphing with Plotly](https://plot.ly/python/)

In [36]:
from IPython.core.display import HTML
import urllib2
HTML(urllib2.urlopen('https://raw.githubusercontent.com/plotly/python-user-guide/css-updates/custom.css').read())